In [84]:
my_path = '/content/drive/MyDrive/additional_package'

!pip install --target=$my_path transformers # transformers 대신에 원하는 패키지 이름을 넣으시면 됩니다



     |████████████████████████████████| 1.2 MB 27.5 MB/s 


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/additional_package')

In [3]:
import os
import re
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm # Progress Bar
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import transformers
from transformers import DistilBertTokenizer, TFDistilBertModel, DistilBertConfig
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
import warnings
from transformers import logging as hf_logging
hf_logging.set_verbosity_error() # Hidding Huggingface Warnings
warnings.filterwarnings("ignore")

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [36]:
test_df = pd.read_csv('/content/drive/MyDrive/통계청/test_preprocessed.csv')
train_preprocessed = pd.read_csv('/content/drive/MyDrive/통계청/train_preprocesed.csv')

In [37]:
# label 빈도 230 미만 짤

# label 빈도 230 미만 짤

In [38]:
except_label = train_preprocessed["label"].value_counts()[train_preprocessed["label"].value_counts() < 230].index
mask = train_preprocessed['label'].isin(except_label)
train_preprocessed = train_preprocessed[~mask].reset_index(drop=True)
train_preprocessed

,text_obj,text_mthd,text_deal,document,label
0,카센터에서,자동차부분정비,타이어오일교환,카센터에서 자동차 부분정비 타이어 오일 교환,S95952
1,상점내에서,일반인을 대상으로,채소 과일판매,상점 내에서 일반인을 대상으로 채소 과일 판매,G47472
2,절단하여사업체에도매,공업용고무를가지고,합성고무도매,절단하여 사업체에도 매 공업용 고무를 가지고 합성고무도 매,G46467
3,영업점에서,일반소비자에게,열쇠잠금장치,영업점에서 일반 소비자에게 열쇠 잠금장치,G47475
4,어린이집,보호자의 위탁을 받아,취학전아동보육,어린이집 보호자의 위탁을 받아 취학 전 아동보육,Q87872
...,...,...,...,...,...
995667,제품입고,워싱,청바지워싱,제품 입고 워싱 청바지 워싱,C13134
995668,현장에서,고객의요청에의해,실내인테리어,현장에서 고객의 요청에 의해 실내 인테리어,F42424
995669,영업점에서,일반소비자에게,여성의류 판매,영업점에서 일반 소비자에게 여성의류 판매,G47474
995670,사업장에서,일반고객을대상으로,필라테스,사업장에서 일반 고객을 대상으로 필라테스,P85856


In [39]:
max_words = []
for row in train_preprocessed['document']:
    max_words.append(len(row.split()))
print(f"The max word count is {np.asarray(max_words).max()}")

The max word count is 25


In [40]:
max_words[:5]

[6, 7, 8, 5, 7]

In [41]:
train_preprocessed.iloc[:,-1]

0         S95952
1         G47472
2         G46467
3         G47475
4         Q87872
           ...  
995667    C13134
995668    F42424
995669    G47474
995670    P85856
995671    I56561
Name: label, Length: 995672, dtype: object

In [48]:
train_preprocessed["label"].value_counts()


I56561    123703
I56562     49756
H49493     49115
S96961     43040
H49492     39176
           ...  
A112         254
C11111       250
C26265       239
N74743       239
J59592       230
Name: label, Length: 169, dtype: int64

In [49]:
# train 데이터를 8:2로 분할 (train set, val set)
train_df = train_preprocessed.sample(frac=0.8, random_state =1)
val_df = train_preprocessed.drop(train_df.index)

In [50]:
#Lable Encode

In [51]:
le = LabelEncoder()
train_df['encoded_label'] = le.fit_transform(train_df['label'])
val_df['encoded_label'] = le.transform(val_df['label'])

In [52]:
val_df

,text_obj,text_mthd,text_deal,document,label,encoded_label
5,철,절삭 용접,카프라배관자재,철 절삭 용접 카프라 배관자재,C29291,51
12,매장에서,일반인 대상으로,의류소매,매장에서 일반인 대상으로 의류 소매,G47474,88
25,영업점에서,일반소비자에게,오징어빵판매,영업점에서 일반 소비자에게 오징어 빵 판매,I56561,102
37,카페에서,접객시설을갖추고,커피 음료수,카페에서 접객시설을 갖추고 커피 음료수,I56562,103
40,여관업,NaN,NaN,여관업,I55551,100
...,...,...,...,...,...,...
995645,음식점에서,접객시설을 갖추고,치킨 맥주,음식점에서 접객시설을 갖추고 치킨 맥주,I56561,102
995651,사업장에서,고객의요청에의해,부동산중개,사업장에서 고객의 요청에 의해 부동산 중개,L68682,119
995664,고객의사업장에서,고객요구의따라,조경건설공사,고객의 사업장에서 고객 요구의 따라 조경건설공사,F41412,67
995665,카본열선 퀼팅구조체,조립 배열 봉제하여,의료용매트리스,카본 열선 퀼팅 구조체 조립 배열 봉제하여 의료용 매트리스,C32320,56


In [53]:
label_dict = (val_df[['label','encoded_label']].drop_duplicates()
              .sort_values(by='encoded_label')
              .reset_index(drop=True)['label']
              .to_dict())

for index, key in label_dict.items():
    print(index, key)

0 A111
1 A112
2 C10101
3 C10102
4 C10103
5 C10104
6 C10106
7 C10107
8 C10108
9 C11111
10 C13131
11 C13132
12 C13134
13 C13139
14 C14141
15 C14143
16 C14144
17 C15151
18 C15152
19 C16161
20 C16162
21 C17172
22 C17179
23 C18181
24 C20201
25 C20202
26 C20203
27 C20204
28 C22221
29 C22222
30 C23231
31 C23232
32 C23233
33 C23239
34 C24241
35 C24242
36 C24243
37 C25251
38 C25259
39 C26261
40 C26262
41 C26263
42 C26264
43 C26265
44 C27271
45 C27272
46 C28281
47 C28283
48 C28284
49 C28285
50 C28289
51 C29291
52 C29292
53 C30302
54 C30303
55 C31311
56 C32320
57 C33331
58 C33333
59 C33339
60 C34340
61 D35351
62 E37370
63 E38381
64 E38382
65 E38383
66 F41411
67 F41412
68 F42421
69 F42422
70 F42423
71 F42424
72 F42425
73 F42426
74 G45451
75 G45452
76 G45453
77 G46461
78 G46462
79 G46463
80 G46464
81 G46465
82 G46466
83 G46467
84 G46468
85 G47471
86 G47472
87 G47473
88 G47474
89 G47475
90 G47476
91 G47477
92 G47478
93 G47479
94 H49492
95 H49493
96 H49494
97 H50501
98 H52521
99 H52529
100 I55551
101

In [54]:
MODEL_NAME = 'monologg/distilkobert'

In [87]:
from kobert_transformers.tokenization_kobert import KoBertTokenizer
tokenizer = KoBertTokenizer.from_pretrained('monologg/distilkobert') # monologg/distilkobert도 동일
tokenizer.tokenize("[CLS] 한국어 모델을 공유합니다. [SEP]")

Downloading:   0%|          | 0.00/363k [00:00<?, ?B/s]

['[CLS]', '▁한국', '어', '▁모델', '을', '▁공유', '합니다', '.', '[SEP]']

In [112]:
tokenizer = KoBertTokenizer.from_pretrained(MODEL_NAME) # Loading the tokenizer

In [113]:
x_train = train_df['document']
y_train = train_df['encoded_label']
x_val = val_df['document']
y_val = val_df['encoded_label']

In [114]:
max_val = 0
for sent in (x_train.tolist() + x_train.tolist() + x_val.tolist()):
    try:
        sent_tok_len = len(tokenizer.tokenize(sent))
        max_val = sent_tok_len if (sent_tok_len > max_val) else max_val
    except:
        pass
    
print(f"The maximum amount of tokens in the dataset is {max_val}")

The maximum amount of tokens in the dataset is 56


max length 와 딥러닝 모델의 input 개수를 27
(Max token length + 2 for special characters [CLS] and [SEP])

In [115]:
MAX_LENGTH = 27
from kobert_transformers.tokenization_kobert import KoBertTokenizer
tokenizer = KoBertTokenizer.from_pretrained(MODEL_NAME,                                              
                                            add_special_tokens=True,
                                            max_length=MAX_LENGTH, 
                                            pad_to_max_length=True) # monologg/distilkobert도 동일

def tokenize(sentences, tokenizer):
    input_ids, input_masks, input_segments = [], [], []
    for sentence in tqdm(sentences):
        inputs = tokenizer.encode_plus(sentence, 
                                       add_special_tokens=True, 
                                       max_length=MAX_LENGTH, 
                                       pad_to_max_length=True, 
                                       return_attention_mask=True, 
                                       return_token_type_ids=True, 
                                       truncation=True)
        input_ids.append(inputs['input_ids'])
        input_masks.append(inputs['attention_mask'])
        input_segments.append(inputs['token_type_ids'])        
        
    return np.asarray(input_ids, dtype='int32'), np.asarray(input_masks, dtype='int32')

In [116]:
X_train = tokenize(x_train, tokenizer)
x_test = test_df['document']
X_test = tokenize(x_test, tokenizer)
X_val = tokenize(x_val, tokenizer)

100%|██████████| 199134/199134 [00:29<00:00, 6724.70it/s]


In [117]:
X_test

(array([[   2, 4617, 7576, ...,    1,    1,    1],
        [   2, 2641, 5452, ...,    1,    1,    1],
        [   2, 4067, 6903, ...,    1,    1,    1],
        ...,
        [   2, 3807, 4085, ...,    1,    1,    1],
        [   2, 2609, 7184, ...,    1,    1,    1],
        [   2, 2609, 7184, ...,    1,    1,    1]], dtype=int32),
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]], dtype=int32))

In [118]:
index_num = 10

print(train_df.iloc[index_num]['label'], '\n')

print(x_train.iloc[index_num], '\n')
temp_tokens = tokenizer.tokenize(x_train.iloc[index_num])
temp_ids = tokenizer.encode(x_train.iloc[index_num])

print('ID\'s', 'Input Tokens', sep='\t')
for i in range(len(temp_ids)):
    if i == 0:
        print(temp_ids[i], '[CLS]', sep='\t')
        continue
    if i == len(temp_ids)-1:
        print(temp_ids[i], '[SEP]', sep='\t')
        break
    print(temp_ids[i], temp_tokens[i-1], sep='\t')


G47479 

인터넷으로 전자상거래 생활용품 

ID's	Input Tokens
2	[CLS]
3794	▁인터넷
7078	으로
4049	▁전자
6527	상
5379	거래
2717	▁생활
7004	용품
3	[SEP]


In [119]:
model = TFDistilBertModel.from_pretrained("distilbert-base-uncased")

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/347M [00:00<?, ?B/s]

In [120]:
config = DistilBertConfig.from_pretrained(MODEL_NAME, output_hidden_states=True, output_attentions=True)
DistilBERT = TFDistilBertModel.from_pretrained(MODEL_NAME, config=config,from_pt=True)


In [126]:
DistilBERT

In [133]:

input_ids_in = tf.keras.layers.Input(shape=(MAX_LENGTH,), name='input_token', dtype='int32')
input_masks_in = tf.keras.layers.Input(shape=(MAX_LENGTH,), name='masked_token', dtype='int32') 

embedding_layer = DistilBERT(input_ids = input_ids_in, attention_mask = input_masks_in)[0]
X = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True))(embedding_layer)
X = tf.keras.layers.GlobalMaxPool1D()(X)
X = tf.keras.layers.Dense(1024, activation='relu')(X)
X = tf.keras.layers.Dropout(0.2)(X)
X = tf.keras.layers.Dense(169, activation='softmax')(X)

model = tf.keras.Model(inputs=[input_ids_in, input_masks_in], outputs = X)

# for layer in model.layers[:3]:
#     layer.trainable = False

model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_token (InputLayer)       [(None, 27)]         0           []                               
                                                                                                  
 masked_token (InputLayer)      [(None, 27)]         0           []                               
                                                                                                  
 tf_distil_bert_model_5 (TFDist  TFBaseModelOutput(l  27803904   ['input_token[0][0]',            
 ilBertModel)                   ast_hidden_state=(N               'masked_token[0][0]']           
                                one, 27, 768),                                                    
                                 hidden_states=((No                                         

In [134]:
output_dir = '/content/drive/MyDrive/통계청/model1'
if not os.path.exists(output_dir): ### If the file directory doesn't already exists,
    os.makedirs(output_dir) ### Make it please

In [135]:
model_checkpoint = ModelCheckpoint(filepath=output_dir+'/weights.{epoch:02d}.hdf5',
                                  save_weights_only=True)

early_stopping = EarlyStopping(patience=3, # Stop after 3 epochs of no improvement
                               monitor='val_loss', # Look at validation_loss
                               min_delta=0, # After 0 change
                               mode='min', # Stop when quantity has stopped decreasing
                               restore_best_weights=False, # Don't Restore the best weights
                               verbose=1) 

reduce_lr = ReduceLROnPlateau(monitor='val_loss', # Look at validation loss
                              min_lr=0.000001, # Lower bound of learning rate
                              patience=1, # Reduce after 1 with little change
                              mode='min', # Stop when quantity has stopped decreasing
                              factor=0.1, # Reduce by a factor of 1/10
                              min_delta=0.01, # Minimumn change needed
                              verbose=1)

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train, 
                    y_train, 
                    epochs = 3,
                    batch_size=16, 
                    validation_data=(X_val, y_val), 
                    callbacks=[model_checkpoint, early_stopping, reduce_lr])

Epoch 1/3
49784/49784 [==============================] - 2027s 41ms/step - loss: 0.8296 - accuracy: 0.7927 - val_loss: 1.0232 - val_accuracy: 0.7422 - lr: 0.0010
Epoch 2/3
42396/49784 [========================>.....] - ETA: 4:32 - loss: 1.1272 - accuracy: 0.7185

In [ ]:
def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
    print("Lowest Validation Loss: epoch {}".format(np.argmin(val_loss)+1))
    print("Highest Validation Accuracy: epoch {}".format(np.argmax(val_acc)+1))

plot_history(history)

In [ ]:
def get_min_val_loss_epoch(history):
    return "0"+str(np.argmin(history.history['val_loss'])+1)

def get_max_val_acc_epoch(history):
    return "0"+str(np.argmax(history.history['val_accuracy'])+1)

In [ ]:
epoch_num = get_max_val_acc_epoch(history)
model.load_weights(output_dir+"/weights."+epoch_num+".hdf5") # Load in model weights

In [ ]:
y_test_probs = model.predict(X_test)

# Turn probabilities into an interger prediction
y_hat = []
for prob in y_test_probs:
    y_hat.append(np.argmax(prob))

In [ ]:
print("Accuracy:", accuracy_score(y_test, y_hat))
print_cf1(y_test, y_hat)

In [ ]:
print(classification_report(y_test, y_hat, target_names=labels_5))

In [ ]:
y_hat_df = pd.DataFrame(columns={'label'})
y_hat_df['label'] = y_hat
y_hat_df.to_csv('/content/drive/MyDrive/통계청/distilkobertresult.csv')


In [ ]:
y_hat_df.to_csv('/content/drive/MyDrive/통계청/distilkobert_0328.csv')